# Multi-variable Linear Regression

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.10.0'

# Hypothesis

## $$ H(x_1, x_2, x_3) = \underline{w_1 x_1 + w_2 x_2 + w_3 x_3} + b $$

## $$ = w_1 x_1 + w_2 x_2 + w_3 x_3 $$ 

## $$ =  \begin{pmatrix} w_{ 1 } & w_{ 2 } & w_{ 3 } \end{pmatrix}\cdot \begin{pmatrix} x_{ 1 } \\ x_{ 2 } \\ x_{ 3 } \end{pmatrix} $$

## $$ H(x_1, x_2, x_3) = WX $$ 



In [4]:
x_data = [
    [1., 0., 3., 0., 5.],
    [0., 2., 0., 4., 0.]
]
y_data  = [1, 2, 3, 4, 5]

W = tf.Variable(tf.random.normal([1, 2], -1.0, 1.0))
b = tf.Variable(tf.random.normal([1], -1.0, 1.0))
print(f"w:{W}")

print(np.array(x_data).shape, np.array(y_data).shape, W.shape, b.shape)

learning_rate = tf.Variable(0.001)

for i in range(100):
   with tf.GradientTape() as tape:
    # (5, 2) * (2, 1) = (5, 1)
    hypothesis = tf.matmul(W, x_data) + b  # w [1, 2] * x [2, 5] = y [1, 5]
    cost = tf.reduce_mean(tf.square(hypothesis - y_data))

    W_grad, b_grad = tape.gradient(cost, [W, b])
    W.assign_sub(learning_rate * W_grad)
    b.assign_sub(learning_rate * b_grad)
    
    if i % 10 == 0:
        print("step: {:3} \t cost: {:5.4f} \t w[0][0]: {:5.4f} \t w[0][1]: {:5.4f} \t b: {:5.4f}".format(
            i, cost.numpy(), W.numpy()[0][0], W.numpy()[0][1], b.numpy()[0]))

w:<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[-2.354416 , -0.2954507]], dtype=float32)>
(2, 5) (5,) (1, 2) (1,)
step:   0 	 cost: 126.7725 	 w[0][0]: -2.2990 	 w[0][1]: -0.2794 	 b: -2.3345
step:  10 	 cost: 94.3316 	 w[0][0]: -1.7892 	 w[0][1]: -0.1286 	 b: -2.1502
step:  20 	 cost: 70.2697 	 w[0][0]: -1.3522 	 w[0][1]: 0.0065 	 b: -1.9900
step:  30 	 cost: 52.4115 	 w[0][0]: -0.9779 	 w[0][1]: 0.1278 	 b: -1.8505
step:  40 	 cost: 39.1482 	 w[0][0]: -0.6571 	 w[0][1]: 0.2366 	 b: -1.7290
step:  50 	 cost: 29.2898 	 w[0][0]: -0.3825 	 w[0][1]: 0.3344 	 b: -1.6231
step:  60 	 cost: 21.9556 	 w[0][0]: -0.1473 	 w[0][1]: 0.4224 	 b: -1.5306
step:  70 	 cost: 16.4938 	 w[0][0]: 0.0541 	 w[0][1]: 0.5015 	 b: -1.4498
step:  80 	 cost: 12.4219 	 w[0][0]: 0.2264 	 w[0][1]: 0.5728 	 b: -1.3791
step:  90 	 cost: 9.3823 	 w[0][0]: 0.3737 	 w[0][1]: 0.6370 	 b: -1.3172


## Test Score


x1 | x2 | x3 | Y
---- | ---- | ----| ----
96 | 91 | 99 | 194
88 | 85 | 82 | 181
78 | 77 | 73 | 177
67 | 66 | 61 | 164
55 | 51 | 53 | 157

# Hypothesis using matrix

$$ [5, 3] \cdot [?, ?] = [5, 1] $$

$$ H(X) = XW + b $$


In [12]:
data = np.array([
    # X1,   X2,    X3,   Y
    [ 96.,  91.,  99., 194. ],
    [ 88.,  85.,  82., 181. ],
    [ 78.,  77.,  73., 177. ],
    [ 67.,  66., 61., 164. ],
    [ 55.,  51.,  53., 157. ]
], dtype=np.float32)

# slice data
X = data[:, :-1]
print(X.shape)
y = data[:, [-1]]
print(y.shape)

# Model
W = tf.Variable(tf.random.normal([3, 1]))
b = tf.Variable(tf.random.normal([1]))

learning_rate = 0.00001

def predict(X):
    return tf.matmul(X, W) + b

print("epoch | cost")

n_epochs = 1000
for i in range(n_epochs):
    # tf.GradientTape() to record the gradient of the cost function
    with tf.GradientTape() as tape:
        cost = tf.reduce_mean((tf.square(predict(X) - y)))

    # Loss함수의 gradient를 계산한다.
    W_grad, b_grad = tape.gradient(cost, [W, b])

    # 파라미터 업데이트 (W and b) -> Optimizer!
    W.assign_sub(learning_rate * W_grad)
    b.assign_sub(learning_rate * b_grad)
    
    if i % 100 == 0:
        print("{:5} | {:10.4f}".format(i, cost.numpy()))

(5, 3)
(5, 1)
epoch | cost
    0 | 51181.0391
  100 |   491.1911
  200 |   487.9503
  300 |   484.8523
  400 |   481.8886
  500 |   479.0529
  600 |   476.3397
  700 |   473.7422
  800 |   471.2551
  900 |   468.8725


## 데이터를 기반으로예측해보자

In [4]:
def predict(X):
    return tf.matmul(X, W) + b # 위쪽에 선언되어 있다.

predict(X).numpy() # prediction, 예측값

array([[214.68604],
       [194.50883],
       [175.49782],
       [149.31918],
       [118.2087 ]], dtype=float32)

In [5]:
# 새로운 데이터에 대한 예측

predict([[ 89.,  95.,  92.],[ 84.,  92.,  85.]]).numpy() 

array([[218.61871],
       [209.49649]], dtype=float32)

### with Tensorflow

In [6]:
data = np.array([
    # X1,   X2,    X3,   y
    [ 96.,  91.,  99., 194. ],
    [ 88.,  85.,  82., 181. ],
    [ 78.,  77.,  73., 177. ],
    [ 67.,  66., 61., 164. ],
    [ 55.,  51.,  53., 157. ]
], dtype=np.float32)

# slice data
X = data[:, :-1]
y = data[:, [-1]] # Raw data

# tf.data generate data from raw data
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.batch(batch_size=1)

model = Sequential([
    layers.Dense(1, activation='linear')
])

model.compile(optimizer='adam', # W assign_sub 
              loss='mse', # mse, mae
              metrics=['mse'])

model.fit(dataset, epochs=1000) # tf.GradientTape 교체가능!

Epoch 1/1000


2023-12-18 11:34:40.542989: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-18 11:34:40.544303: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 1s 6ms/step - loss: 110887.3125 - mse: 110887.3125
Epoch 2/1000
5/5 [==============================] - 0s 6ms/step - loss: 110162.8984 - mse: 110162.8984
Epoch 3/1000
5/5 [==============================] - 0s 6ms/step - loss: 109450.3516 - mse: 109450.3516
Epoch 4/1000
5/5 [==============================] - 0s 6ms/step - loss: 108740.8750 - mse: 108740.8750
Epoch 5/1000
5/5 [==============================] - 0s 6ms/step - loss: 108033.7344 - mse: 108033.7344
Epoch 6/1000
5/5 [==============================] - 0s 6ms/step - loss: 107328.8984 - mse: 107328.8984
Epoch 7/1000
5/5 [==============================] - 0s 6ms/step - loss: 106626.4219 - mse: 106626.4219
Epoch 8/1000
5/5 [==============================] - 0s 6ms/step - loss: 105926.4531 - mse: 105926.4531
Epoch 9/1000
5/5 [==============================] - 0s 6ms/step - loss: 105229.1094 - mse: 105229.1094
Epoch 10/1000
5/5 [==============================] - 0s 12ms/step - loss: 104534.5078 

In [7]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [8]:
test_loss, test_mae = model.evaluate(X, y, verbose=0)
print('Test MSE:', test_mae) 

2023-12-18 11:35:15.602518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Test MSE: 446.7474060058594


In [9]:
for x, y in dataset:
    print(x)
    print(y)
    print(model(x))
    break

tf.Tensor([[96. 91. 99.]], shape=(1, 3), dtype=float32)
tf.Tensor([[194.]], shape=(1, 1), dtype=float32)
tf.Tensor([[216.72923]], shape=(1, 1), dtype=float32)
